## Sending data to Kafka server

This notebook uses the [Python client for the Apache Kafka distributed stream processing system](http://kafka-python.readthedocs.io/en/master/index.html) to send messages to a Kafka cluster. 

In this example, the messages are data generated from a linear model with $n$ input variables, i.e., 

$$
y =x^T \beta +w
$$
with $x, \beta \in \mathbb{R}^n$, and $y, w \in \mathbb{R}$. $w$ is Gaussian noise.

Messages are sent every $interval$ seconds. They are list of size (n+2) where:
* First element is the counter 
* Second element is $y$ 
* Third to last elements are $x$ values (size $n$)


In [59]:
from kafka import KafkaProducer
import time
import numpy as np
import pandas as pd

In [60]:
#Takes about one minute to load
data=pd.read_csv("../../data/data.conv.txt.gz",header=None,sep=" ")
data.columns=["Date","Hour","Sensor","Value","Voltage"]
data=data.sort_values(['Date','Hour']).reset_index(drop=True)

data['datetime']=pd.to_datetime(data.Date+' '+data.Hour)
data['relative_datetime']=data['datetime']-data['datetime'][0]
data['seconds']=data['relative_datetime'].dt.total_seconds()

sensorId_type=data.Sensor.str.split("-",expand=True)
sensorId_type.columns=['SensorId','Type']
data['SensorId']=sensorId_type['SensorId'].astype(int)
data['Type']=sensorId_type['Type'].astype(int)

In [61]:
#Drop features not needed for the simulation
data=data.drop(['datetime','relative_datetime','Sensor','Date','Hour','Voltage'],axis=1)

In [62]:
data[:5]

,Value,seconds,SensorId,Type
0,17.6364,0.000,41,0
1,16.6956,0.007,44,0
2,45.7037,0.092,41,1
3,2.3000,0.237,44,2
4,47.9942,0.285,44,1


In [63]:
temp=data[(data.SensorId==1) & (data.Type==0)]
temp=temp.reset_index(drop=True)

In [64]:
temp[:5]

,Value,seconds,SensorId,Type
0,19.2436,79.124,1,0
1,19.2240,169.155,1,0
2,19.2142,200.931,1,0
3,19.1848,264.276,1,0
4,19.1946,321.270,1,0


In [75]:
temp_train = temp[temp.seconds < 8*86400]
temp_test = temp[(temp.seconds >= 8*86400) & (temp.seconds < 9*86400)]

temp_test[:10]

,Value,seconds,SensorId,Type
16592,22.6932,691234.125,1,0
16593,22.6834,691262.504,1,0
16594,22.6834,691302.766,1,0
16595,22.6638,691327.934,1,0
16596,22.6638,691364.159,1,0
16597,22.6540,691384.790,1,0
16598,22.6540,691415.179,1,0
16599,22.6540,691450.694,1,0
16600,22.6736,691476.007,1,0
16601,22.6736,691507.201,1,0


In [78]:
interval = 3600
last_temp = temp_test.iloc[-1].seconds
time_bins = int(last_temp / interval) + 1
time_bins

216

In [82]:
def rollup(temp, interval, day_begin=0):
    # roll up data according to a time interval
    first_temp = day_begin * 86400
    last_temp = temp.iloc[-1].seconds
    time_bins = int((last_temp-first_temp)/interval) + 1
    temp_sync = np.zeros((time_bins, 5)) # Value, bin, SensorID, Type, number of elem in bin
    
    for i in range(len(temp)):
        data = temp.iloc[i]
        bin = int((data.seconds-first_temp)/interval)
        n = temp_sync[bin][-1] + 1
        new_value = (temp_sync[bin][0] * (n-1) + data.Value)/n # running avg
        temp_sync[bin][0] = new_value
        temp_sync[bin][1] = bin
        temp_sync[bin][2] = data.SensorId
        temp_sync[bin][3] = data.Type
        temp_sync[bin][4] = n
    return temp_sync


def delta_max(temp):
    # evaluate the maximum time interval between two data
    maxT = 0
    for i in range(1,len(temp)):
        time_now = temp.iloc[i].seconds
        time_before = temp.iloc[i-1].seconds
        delta = time_now - time_before
        if delta > maxT:
            maxT = delta
    return maxT


def interpolate(temp_sync, K):
    # interpolate missing data in rolled up data using the data of K neighbours
    # rolled up data referes to the output of rollup function
    # K is K/2 past data and K/2 futur data
    for bin in range(len(temp_sync)):
        # if no value provided
        left_range = max(0, bin-K/2)
        right_range = min(len(temp_sync), bin+K/2)
        if temp_sync[bin][-1] == 0:
            bins = []
            vals = []
            for n in range(int(left_range), int(right_range)):
                if temp_sync[n][-1] != 0:
                    bins.append(n)
                    vals.append(temp_sync[n][0])
            val_interp = np.interp(bin, bins, vals)
            temp_sync[bin][0] = val_interp
            temp_sync[bin][1] = bin
            temp_sync[bin][2] = 1
            temp_sync[bin][3] = 0
            temp_sync[bin][4] = 1
    return temp_sync

In [67]:
delta_max_train = delta_max(temp_train)
delta_max_test = delta_max(temp_test)
print(delta_max_train)
print(delta_max_test)

339.348
210.811


In [83]:
# Create a cleaned temperature vector:
# we want one data every interval, this is not always the case

interval = 3600 # seconds
temp_train_sync = rollup(temp_train, interval)
temp_test_sync = rollup(temp_test, interval, day_begin=8)

print('Number of intervals of length %d in train set: %d' %(interval, len(temp_train_sync)))
print('Number of intervals of length %d in test set: %d' %(interval, len(temp_test_sync)))

Number of intervals of length 3600 in train set: 192
Number of intervals of length 3600 in test set: 24


In [84]:
temp_test_sync[:]

array([[  22.56749615,    0.        ,    1.        ,    0.        ,  104.        ],
       [  22.31450737,    1.        ,    1.        ,    0.        ,   95.        ],
       [  22.10268257,    2.        ,    1.        ,    0.        ,  109.        ],
       [  21.85943137,    3.        ,    1.        ,    0.        ,  102.        ],
       [  21.66275579,    4.        ,    1.        ,    0.        ,   95.        ],
       [  21.45400374,    5.        ,    1.        ,    0.        ,  107.        ],
       [  21.33437204,    6.        ,    1.        ,    0.        ,   93.        ],
       [  24.07922947,    7.        ,    1.        ,    0.        ,   95.        ],
       [  24.81149333,    8.        ,    1.        ,    0.        ,  105.        ],
       [  25.84166304,    9.        ,    1.        ,    0.        ,   92.        ],
       [  23.69314588,   10.        ,    1.        ,    0.        ,   85.        ],
       [  22.46092874,   11.        ,    1.        ,    0.        ,   87.   

In [85]:
K = 20 # interpolation memory, choose according to interval (and eventually maxT)
temp_train_sync = interpolate(temp_train_sync, K)
temp_test_sync = interpolate(temp_test_sync, K)
temp_train_sync[:10]

array([[  19.07656765,    0.        ,    1.        ,    0.        ,   68.        ],
       [  18.84410588,    1.        ,    1.        ,    0.        ,  119.        ],
       [  18.47977167,    2.        ,    1.        ,    0.        ,  120.        ],
       [  18.21451695,    3.        ,    1.        ,    0.        ,  118.        ],
       [  17.97632712,    4.        ,    1.        ,    0.        ,  118.        ],
       [  17.73514746,    5.        ,    1.        ,    0.        ,  118.        ],
       [  17.55193333,    6.        ,    1.        ,    0.        ,  105.        ],
       [  17.6034451 ,    7.        ,    1.        ,    0.        ,  102.        ],
       [  18.90315652,    8.        ,    1.        ,    0.        ,  115.        ],
       [  21.10388889,    9.        ,    1.        ,    0.        ,  126.        ]])

In [23]:
producer = KafkaProducer(bootstrap_servers='kafka1:19092,kafka2:29092,kafka3:39092')

In [87]:
# Feeding training data:

n = 24 # number of parameters
# n parameters -> n last temperatures
for t in range(n, len(temp_train_sync)):
    x = temp_train_sync[t-n:t, 0] # n last temperatures
    y = temp_train_sync[t, 0] # current temperature
    message = np.array2string(np.append([t,y],x),separator=",")
    print(message)
    producer.send('RLSTrain', message.encode())
    time.sleep(1)
    

[ 24.        , 18.35869062, 19.07656765, 18.84410588, 18.47977167,
  18.21451695, 17.97632712, 17.73514746, 17.55193333, 17.6034451 ,
  18.90315652, 21.10388889, 21.90129123, 22.79103934, 22.70924928,
  22.72514351, 22.4845082 , 21.96226667, 21.12933   , 20.45813684,
  20.17318993, 19.76572222, 19.62323605, 19.35175766, 19.1163298 ,
  18.9141589 ]
[ 25.        , 18.19215484, 18.84410588, 18.47977167, 18.21451695,
  17.97632712, 17.73514746, 17.55193333, 17.6034451 , 18.90315652,
  21.10388889, 21.90129123, 22.79103934, 22.70924928, 22.72514351,
  22.4845082 , 21.96226667, 21.12933   , 20.45813684, 20.17318993,
  19.76572222, 19.62323605, 19.35175766, 19.1163298 , 18.9141589 ,
  18.35869062]
[ 26.        , 18.00967761, 18.47977167, 18.21451695, 17.97632712,
  17.73514746, 17.55193333, 17.6034451 , 18.90315652, 21.10388889,
  21.90129123, 22.79103934, 22.70924928, 22.72514351, 22.4845082 ,
  21.96226667, 21.12933   , 20.45813684, 20.17318993, 19.76572222,
  19.62323605, 19.35175766, 19.1

[ 48.        , 19.10024651, 18.35869062, 18.19215484, 18.00967761,
  17.92826957, 17.74824444, 17.49534426, 17.31932658, 20.63867342,
  21.19837333, 21.816345  , 22.34254375, 23.2945    , 24.7204    ,
  25.36307447, 25.75941928, 25.14113684, 25.22057612, 25.5548    ,
  22.01404819, 21.15738272, 20.70770097, 20.33529773, 19.91047229,
  19.53564   ]
[ 49.        , 18.69579241, 18.19215484, 18.00967761, 17.92826957,
  17.74824444, 17.49534426, 17.31932658, 20.63867342, 21.19837333,
  21.816345  , 22.34254375, 23.2945    , 24.7204    , 25.36307447,
  25.75941928, 25.14113684, 25.22057612, 25.5548    , 22.01404819,
  21.15738272, 20.70770097, 20.33529773, 19.91047229, 19.53564   ,
  19.10024651]
[ 50.        , 18.33698605, 18.00967761, 17.92826957, 17.74824444,
  17.49534426, 17.31932658, 20.63867342, 21.19837333, 21.816345  ,
  22.34254375, 23.2945    , 24.7204    , 25.36307447, 25.75941928,
  25.14113684, 25.22057612, 25.5548    , 22.01404819, 21.15738272,
  20.70770097, 20.33529773, 19.9

[ 72.        , 19.95466977, 19.10024651, 18.69579241, 18.33698605,
  18.07128941, 17.78186506, 17.52176279, 17.39725747, 21.66777412,
  23.8606    , 25.22729459, 24.28940811, 23.91094545, 24.34146076,
  24.37461707, 24.33637317, 24.0064    , 23.22962105, 21.77853333,
  21.53803789, 21.23664651, 20.8977    , 20.65075217, 20.41482564,
  20.15226512]
[ 73.        , 19.71697303, 18.69579241, 18.33698605, 18.07128941,
  17.78186506, 17.52176279, 17.39725747, 21.66777412, 23.8606    ,
  25.22729459, 24.28940811, 23.91094545, 24.34146076, 24.37461707,
  24.33637317, 24.0064    , 23.22962105, 21.77853333, 21.53803789,
  21.23664651, 20.8977    , 20.65075217, 20.41482564, 20.15226512,
  19.95466977]
[ 74.        , 19.24476667, 18.33698605, 18.07128941, 17.78186506,
  17.52176279, 17.39725747, 21.66777412, 23.8606    , 25.22729459,
  24.28940811, 23.91094545, 24.34146076, 24.37461707, 24.33637317,
  24.0064    , 23.22962105, 21.77853333, 21.53803789, 21.23664651,
  20.8977    , 20.65075217, 20.4

[ 96.        , 20.34642667, 19.95466977, 19.71697303, 19.24476667,
  18.91405672, 18.55592955, 18.32485   , 18.29778333, 22.92158261,
  25.65116667, 25.61773253, 25.455085  , 24.34906591, 23.39107529,
  24.37312632, 24.41183548, 24.21837534, 23.98707222, 24.10700506,
  22.96150811, 22.1855122 , 21.70169565, 21.40565618, 21.12137188,
  20.79083947]
[ 97.        , 19.79330263, 19.71697303, 19.24476667, 18.91405672,
  18.55592955, 18.32485   , 18.29778333, 22.92158261, 25.65116667,
  25.61773253, 25.455085  , 24.34906591, 23.39107529, 24.37312632,
  24.41183548, 24.21837534, 23.98707222, 24.10700506, 22.96150811,
  22.1855122 , 21.70169565, 21.40565618, 21.12137188, 20.79083947,
  20.34642667]
[ 98.        , 19.3276    , 19.24476667, 18.91405672, 18.55592955,
  18.32485   , 18.29778333, 22.92158261, 25.65116667, 25.61773253,
  25.455085  , 24.34906591, 23.39107529, 24.37312632, 24.41183548,
  24.21837534, 23.98707222, 24.10700506, 22.96150811, 22.1855122 ,
  21.70169565, 21.40565618, 21.1

[ 119.        ,  20.51580563,  20.79083947,  20.34642667,  19.79330263,
   19.3276    ,  19.00479706,  18.6586625 ,  18.40294795,  18.26486   ,
   22.71763467,  24.19614468,  25.80321348,  24.867624  ,  25.16528267,
   26.28411884,  26.24563099,  25.10367869,  23.66493125,  22.67752   ,
   21.74015   ,  21.5428411 ,  21.40600769,  21.15570423,  20.93464444,
   20.72498919]
[ 120.        ,  20.33604211,  20.34642667,  19.79330263,  19.3276    ,
   19.00479706,  18.6586625 ,  18.40294795,  18.26486   ,  22.71763467,
   24.19614468,  25.80321348,  24.867624  ,  25.16528267,  26.28411884,
   26.24563099,  25.10367869,  23.66493125,  22.67752   ,  21.74015   ,
   21.5428411 ,  21.40600769,  21.15570423,  20.93464444,  20.72498919,
   20.51580563]
[ 121.        ,  20.08509333,  19.79330263,  19.3276    ,  19.00479706,
   18.6586625 ,  18.40294795,  18.26486   ,  22.71763467,  24.19614468,
   25.80321348,  24.867624  ,  25.16528267,  26.28411884,  26.24563099,
   25.10367869,  23.66493125,  2

[ 141.        ,  20.90035211,  20.93464444,  20.72498919,  20.51580563,
   20.33604211,  20.08509333,  19.83344638,  19.576016  ,  19.31352432,
   19.12548421,  19.07334328,  23.57236316,  25.60962703,  24.14192683,
   24.18186667,  23.46484932,  23.03740822,  24.19694177,  24.09938049,
   23.51888451,  22.89725479,  22.08723333,  21.76748539,  21.49690824,
   21.2232    ]
[ 142.        ,  20.74977353,  20.72498919,  20.51580563,  20.33604211,
   20.08509333,  19.83344638,  19.576016  ,  19.31352432,  19.12548421,
   19.07334328,  23.57236316,  25.60962703,  24.14192683,  24.18186667,
   23.46484932,  23.03740822,  24.19694177,  24.09938049,  23.51888451,
   22.89725479,  22.08723333,  21.76748539,  21.49690824,  21.2232    ,
   20.90035211]
[ 143.        ,  20.60529091,  20.51580563,  20.33604211,  20.08509333,
   19.83344638,  19.576016  ,  19.31352432,  19.12548421,  19.07334328,
   23.57236316,  25.60962703,  24.14192683,  24.18186667,  23.46484932,
   23.03740822,  24.19694177,  2

[ 163.        ,  22.79458333,  21.49690824,  21.2232    ,  20.90035211,
   20.74977353,  20.60529091,  20.25070123,  19.93063158,  19.71593721,
   19.61253415,  19.49388539,  19.3582    ,  19.13366957,  19.24483908,
   20.97808605,  22.89508   ,  24.86171683,  26.24376579,  27.04156627,
   27.1719225 ,  26.43896632,  25.96045393,  25.18084634,  24.48176712,
   23.42514545]
[ 164.        ,  22.40312   ,  21.2232    ,  20.90035211,  20.74977353,
   20.60529091,  20.25070123,  19.93063158,  19.71593721,  19.61253415,
   19.49388539,  19.3582    ,  19.13366957,  19.24483908,  20.97808605,
   22.89508   ,  24.86171683,  26.24376579,  27.04156627,  27.1719225 ,
   26.43896632,  25.96045393,  25.18084634,  24.48176712,  23.42514545,
   22.79458333]
[ 165.        ,  22.07819024,  20.90035211,  20.74977353,  20.60529091,
   20.25070123,  19.93063158,  19.71593721,  19.61253415,  19.49388539,
   19.3582    ,  19.13366957,  19.24483908,  20.97808605,  22.89508   ,
   24.86171683,  26.24376579,  2

[ 185.        ,  26.22587209,  24.48176712,  23.42514545,  22.79458333,
   22.40312   ,  22.07819024,  21.80308602,  21.49095402,  21.1919    ,
   20.93192911,  20.61844516,  20.453312  ,  20.22656579,  20.019025  ,
   19.84838511,  20.0703549 ,  21.89507273,  24.09658333,  25.8648475 ,
   27.26102558,  28.05960263,  28.55385128,  28.15159583,  27.57529383,
   27.141175  ]
[ 186.        ,  25.10480825,  23.42514545,  22.79458333,  22.40312   ,
   22.07819024,  21.80308602,  21.49095402,  21.1919    ,  20.93192911,
   20.61844516,  20.453312  ,  20.22656579,  20.019025  ,  19.84838511,
   20.0703549 ,  21.89507273,  24.09658333,  25.8648475 ,  27.26102558,
   28.05960263,  28.55385128,  28.15159583,  27.57529383,  27.141175  ,
   26.22587209]
[ 187.        ,  24.22132079,  22.79458333,  22.40312   ,  22.07819024,
   21.80308602,  21.49095402,  21.1919    ,  20.93192911,  20.61844516,
   20.453312  ,  20.22656579,  20.019025  ,  19.84838511,  20.0703549 ,
   21.89507273,  24.09658333,  2

In [89]:
# Feeding testing data

initial_conditions = temp_train_sync[-n:, 0]
for t in range(len(temp_test_sync)):
    x = initial_conditions
    y = temp_test_sync[t, 0]
    message = np.array2string(np.append([t,y],x),separator=",")
    print(message)
    producer.send('RLSTest', message.encode())
    time.sleep(1)
    

[  0.        , 22.56749615, 21.1919    , 20.93192911, 20.61844516,
  20.453312  , 20.22656579, 20.019025  , 19.84838511, 20.0703549 ,
  21.89507273, 24.09658333, 25.8648475 , 27.26102558, 28.05960263,
  28.55385128, 28.15159583, 27.57529383, 27.141175  , 26.22587209,
  25.10480825, 24.22132079, 23.65802885, 23.27640208, 23.03248632,
  22.81039588]
[  1.        , 22.31450737, 21.1919    , 20.93192911, 20.61844516,
  20.453312  , 20.22656579, 20.019025  , 19.84838511, 20.0703549 ,
  21.89507273, 24.09658333, 25.8648475 , 27.26102558, 28.05960263,
  28.55385128, 28.15159583, 27.57529383, 27.141175  , 26.22587209,
  25.10480825, 24.22132079, 23.65802885, 23.27640208, 23.03248632,
  22.81039588]
[  2.        , 22.10268257, 21.1919    , 20.93192911, 20.61844516,
  20.453312  , 20.22656579, 20.019025  , 19.84838511, 20.0703549 ,
  21.89507273, 24.09658333, 25.8648475 , 27.26102558, 28.05960263,
  28.55385128, 28.15159583, 27.57529383, 27.141175  , 26.22587209,
  25.10480825, 24.22132079, 23.6

In [3]:
i=0 #Initialise counter

n=10   # number of inputs

beta=np.zeros(n) ## first and last parameters are 1, others are zeros
beta[1]=1   
beta[-1]=1
beta.shape=(n,1)


#Loop for sending messages to Kafka with the topic dataLinearModel

while True:
    x=np.random.rand(1,n)[0]
    y=float(x.dot(beta))+0.1*np.random.rand(1)[0] ## y =x^T beta +w
    message=np.array2string(np.append([i,y],x),separator=",") 
    print(message)
    producer.send('dataLinearModel', message.encode())
    i=i+1
    time.sleep(1)

[ 0.        , 0.95772618, 0.67060354, 0.24211019, 0.17263733, 0.79885759,
  0.48056794, 0.98570543, 0.1627009 , 0.58327203, 0.78245165, 0.68484422]
[ 1.        , 0.65940146, 0.10026097, 0.05903539, 0.22909641, 0.32563214,
  0.6809396 , 0.60368021, 0.39138901, 0.26470633, 0.22869574, 0.52838563]
[ 2.        , 0.2607595 , 0.63829029, 0.13070341, 0.41263292, 0.54827117,
  0.04545682, 0.82817345, 0.67135199, 0.6920729 , 0.5594482 , 0.06165693]
[ 3.        , 0.95416609, 0.04906261, 0.12341107, 0.1070854 , 0.00331074,
  0.36320009, 0.2750774 , 0.57627341, 0.03799513, 0.60491055, 0.77024075]
[ 4.        , 1.7365181 , 0.92561066, 0.80130513, 0.36961445, 0.1497974 ,
  0.0540831 , 0.53595227, 0.70130951, 0.77182248, 0.23849375, 0.88075003]
[ 5.        , 1.21626752, 0.92572199, 0.6323603 , 0.83326384, 0.66514895,
  0.12938035, 0.99302658, 0.18598089, 0.5205909 , 0.48324244, 0.51460192]
[ 6.        , 0.86638299, 0.34990381, 0.54791102, 0.95241497, 0.70053108,
  0.23466649, 0.05525377, 0.45685857, 

[ 49.        ,  1.87313449,  0.62510852,  0.99297458,  0.14825713,
   0.33426   ,  0.96796691,  0.38571584,  0.14563271,  0.68380523,
   0.0765593 ,  0.87961825]
[ 50.        ,  0.85262817,  0.37115393,  0.79027126,  0.12221132,
   0.52871574,  0.18528548,  0.94761125,  0.67077707,  0.56343033,
   0.73476217,  0.05936989]
[ 51.        ,  1.53177351,  0.07192952,  0.8356925 ,  0.30456939,
   0.72036634,  0.60152491,  0.10571554,  0.53027396,  0.45361879,
   0.83916123,  0.68076368]
[ 52.        ,  0.54462778,  0.14085798,  0.13559422,  0.60213847,
   0.81927175,  0.40551726,  0.85744463,  0.39721184,  0.37305844,
   0.62073097,  0.33708896]
[  5.30000000e+01,  5.13963328e-01,  4.61748724e-01,  8.03057483e-03,
   3.47863740e-01,  4.84717892e-01,  2.52756041e-01,  7.83358292e-01,
   3.77895161e-01,  4.84854538e-01,  6.85209293e-01,  4.11295530e-01]
[  5.40000000e+01,  6.12267499e-01,  6.67857444e-01,  3.05062782e-03,
   8.59376114e-02,  2.48637159e-01,  7.93277057e-01,  4.25576201e-01,
  

[ 95.        ,  0.74256141,  0.7398143 ,  0.41258847,  0.82803955,
   0.15263898,  0.21054319,  0.42126611,  0.95203626,  0.10866459,
   0.51548958,  0.2905914 ]
[ 96.        ,  1.6696113 ,  0.98821607,  0.77123873,  0.67754367,
   0.43592821,  0.59097779,  0.20514365,  0.47219884,  0.28216909,
   0.4722199 ,  0.84666987]
[  9.70000000e+01,  4.85469055e-01,  6.60786921e-01,  2.33056135e-01,
   3.84478482e-01,  3.91092594e-02,  7.71698443e-01,  1.49730783e-01,
   2.20663948e-01,  7.69490805e-01,  6.71920478e-01,  1.69012289e-01]
[  9.80000000e+01,  1.36440073e+00,  1.98287538e-01,  3.30879059e-01,
   3.45809287e-01,  2.19958074e-01,  1.17128169e-01,  3.16137321e-01,
   7.34358494e-02,  1.84843309e-01,  5.56264035e-01,  9.55274517e-01]
[ 99.        ,  0.78407331,  0.36046046,  0.11776012,  0.67550267,
   0.79697578,  0.86638062,  0.19087129,  0.92443062,  0.22827448,
   0.6916496 ,  0.61069823]
[  1.00000000e+02,  1.05427074e+00,  2.23994077e-01,  2.22770007e-01,
   6.05842881e-01,  3.50

[ 137.        ,   1.56186691,   0.99772438,   0.73882414,   0.83229368,
    0.97996755,   0.77968494,   0.59510207,   0.34350337,   0.97291179,
    0.81865215,   0.78996039]
[  1.38000000e+02,  7.57344521e-01,  1.84121855e-03,  6.36404205e-01,
   8.86699789e-01,  9.81942860e-01,  3.03023302e-01,  4.47183902e-01,
   3.96710388e-01,  4.58223461e-01,  3.27556420e-01,  5.25036817e-02]
[  1.39000000e+02,  4.98088612e-01,  4.56647121e-01,  3.91178471e-01,
   1.91155223e-01,  1.05275659e-01,  2.12562195e-01,  3.22837434e-02,
   3.48470711e-01,  8.48370513e-01,  8.33550996e-01,  1.06781542e-01]
[  1.40000000e+02,  1.92212795e+00,  9.98387534e-01,  9.29409010e-01,
   6.17675495e-01,  6.81671152e-01,  2.64679670e-03,  2.80075024e-01,
   2.14498159e-02,  3.06135621e-01,  2.89395416e-01,  9.45287703e-01]
[  1.41000000e+02,  1.01230413e+00,  8.83800860e-01,  1.12125637e-01,
   7.70669374e-01,  1.91134872e-01,  8.47097335e-01,  5.38594024e-01,
   8.12080883e-01,  9.81414722e-01,  4.82912664e-01,  8.

[  1.78000000e+02,  1.82875554e+00,  6.85009784e-01,  9.62205070e-01,
   2.33256374e-02,  4.96188875e-01,  1.97638018e-01,  2.90406817e-01,
   7.27624227e-01,  4.20159381e-01,  7.42523967e-01,  7.83997807e-01]
[ 179.        ,   0.81341134,   0.54902644,   0.54997082,   0.704293  ,
    0.26121434,   0.73872776,   0.93690926,   0.64383099,   0.57384691,
    0.58668034,   0.23413189]
[  1.80000000e+02,  8.29697063e-01,  1.43703938e-01,  4.55591019e-01,
   9.31675835e-01,  5.55087904e-01,  7.06814862e-02,  6.72866323e-01,
   4.51644377e-01,  8.61214737e-01,  6.61000725e-01,  3.13623451e-01]
[ 181.        ,   0.92105487,   0.58577252,   0.55747535,   0.18783516,
    0.85034097,   0.82093457,   0.31894866,   0.50293326,   0.29657657,
    0.7351163 ,   0.31786376]
[  1.82000000e+02,  1.33262804e+00,  6.22761781e-01,  5.73983404e-01,
   3.46354217e-01,  5.98646880e-02,  4.59793514e-01,  9.72061587e-01,
   3.53080028e-01,  2.44058206e-01,  6.63953027e-01,  6.71737402e-01]
[  1.83000000e+02,  1.

[  2.19000000e+02,  1.52232424e+00,  2.94435038e-01,  5.17974212e-01,
   7.31628761e-01,  8.21507627e-03,  1.37696962e-01,  8.63706904e-01,
   2.15442942e-01,  4.10482911e-01,  5.34053617e-01,  9.30558840e-01]
[  2.20000000e+02,  8.93981474e-01,  6.78031313e-01,  4.98971550e-01,
   8.43419256e-01,  5.17246108e-02,  4.22236392e-03,  6.78908640e-01,
   4.08572853e-01,  2.80855511e-01,  8.77346136e-01,  3.73867061e-01]
[  2.21000000e+02,  1.02867200e+00,  1.37114082e-01,  3.74474724e-01,
   1.39824653e-01,  6.30201472e-01,  5.27514491e-02,  1.55546721e-01,
   3.53721296e-01,  7.49433302e-01,  6.22094467e-01,  5.58719219e-01]
[  2.22000000e+02,  1.41758510e+00,  6.78739913e-02,  4.29769880e-01,
   8.92242018e-01,  3.11614812e-02,  8.19866398e-01,  2.74089879e-01,
   3.93933653e-01,  7.73496980e-01,  3.51457460e-01,  9.67317940e-01]
[  2.23000000e+02,  1.21644562e+00,  7.67136369e-02,  8.74078533e-01,
   4.29372282e-01,  9.39368616e-01,  6.98425433e-01,  1.37329695e-01,
   9.00577398e-01,  

[  2.59000000e+02,  9.21804610e-01,  8.81025149e-01,  5.98463907e-02,
   1.25308028e-01,  7.97221413e-01,  5.26468124e-01,  5.97221169e-01,
   7.56180103e-01,  1.11127410e-01,  3.48878014e-01,  8.59532760e-01]
[  2.60000000e+02,  1.10892291e+00,  8.38355992e-01,  6.71369543e-01,
   8.43517127e-01,  6.92606033e-01,  3.25365642e-01,  1.19906416e-01,
   8.72460866e-01,  5.40435782e-01,  3.17458219e-01,  3.70615578e-01]
[  2.61000000e+02,  1.01373696e+00,  6.32787331e-02,  6.94373043e-01,
   7.28604506e-01,  7.59644155e-01,  2.43176596e-01,  6.98170100e-01,
   3.44032375e-01,  6.18245322e-01,  6.62289125e-01,  2.84650567e-01]
[  2.62000000e+02,  1.02990244e+00,  8.91542202e-01,  7.88932814e-01,
   1.52445833e-01,  4.89390947e-01,  8.37448099e-01,  3.13640470e-01,
   3.51954809e-01,  2.68490924e-01,  9.34797539e-01,  1.53344422e-01]
[  2.63000000e+02,  4.45038113e-01,  2.17660932e-01,  2.79841010e-01,
   1.08898440e-01,  8.66654122e-01,  6.07488436e-02,  5.10911444e-01,
   4.48856088e-01,  

[  2.99000000e+02,  1.06174472e+00,  3.03100166e-01,  1.47760876e-01,
   1.20282542e-01,  7.12303989e-01,  4.19277850e-01,  8.88124125e-01,
   8.63633713e-01,  6.92813643e-01,  2.55863371e-01,  8.88044353e-01]
[  3.00000000e+02,  7.96047208e-01,  8.12009039e-01,  2.47553377e-02,
   2.28740953e-01,  5.05999549e-01,  1.22879474e-01,  4.61715299e-01,
   4.44393358e-01,  9.29285284e-01,  1.26860686e-01,  7.14768131e-01]
[  3.01000000e+02,  1.75520054e+00,  8.53669787e-01,  7.44129656e-01,
   9.53786481e-01,  8.46402510e-01,  5.84660562e-01,  5.61710609e-01,
   2.11651644e-01,  2.45065390e-01,  4.07838127e-02,  9.87382149e-01]
[  3.02000000e+02,  8.45640460e-01,  5.34555820e-01,  5.54810256e-01,
   6.47091176e-01,  3.13049701e-01,  5.92483266e-01,  5.00259298e-01,
   8.89477920e-01,  5.73420687e-01,  8.58084435e-01,  2.58458725e-01]
[  3.03000000e+02,  1.76698678e+00,  6.86137209e-01,  8.77545669e-01,
   1.78887386e-01,  8.65480733e-01,  6.72477197e-01,  6.64758132e-01,
   3.52301421e-01,  

[  3.39000000e+02,  4.72944846e-01,  7.99646771e-01,  4.03747488e-01,
   1.20106340e-02,  5.08223959e-01,  1.74215977e-01,  3.60462012e-02,
   1.91199483e-01,  1.59315988e-01,  7.12895586e-01,  3.05595571e-02]
[  3.40000000e+02,  4.57426466e-01,  4.01728911e-01,  3.70136246e-01,
   5.95981926e-01,  1.91499102e-02,  9.17663702e-02,  1.58054703e-01,
   6.46276694e-01,  6.30856149e-01,  2.83864794e-01,  8.13679283e-02]


KeyboardInterrupt: 